# Fisheries Training Part 1 - Harvest Optimization and MOEA Diagnostics

This is the second post in a training series, studying decision making under deep uncertainty within the context of a complex harvested predator-prey fishery. 

The first post in this training series, [Fisheries Training 0: Exploring Predator-Prey Dynamics, can be found here](https://waterprogramming.wordpress.com/2022/07/11/__trashed-3/). 

The GitHub repository, containing all of the source code used throughout this series, is available [here](https://github.com/TrevorJA/harvested_predator_prey_system_tutorial_series). 

## Harvested predator-prey model

Re-vist the ODEs here.



## Harvest strategy objectives (Trevor)


#### Objective #1: 


#### Objective #2:


#### Objective #3:


#### Objective #4:



In [ ]:
# Import common libraries
import numpy as np
import matplotlib.plot as plt


In [ ]:
# Code for the fish game here, with objective outputs

def fish_game():
    
    return


## PyBorg (Lillian)

Text body.

Mention how it is paired with Platypus as well

### Import all libraries
All functions required for this post can be found in the fish_game_functions.py file. This code is adapted from [Antonia Hadjimichael's original post](https://waterprogramming.wordpress.com/2021/08/13/introduction-to-pyborg-basic-setup-and-running/) on exploring the Fisheries Game dynamics using PyBorg

In [7]:
# import all required libraries
import sys
sys.path.append('C:/Users/lbl59/AppData/Local/Programs/Python/Python39/Lib/site-packages/platypus')
from platypus import Problem, Real, Hypervolume, Generator
from pyborg import BorgMOEA
from fish_game_functions import (fish_game_5_objs, fish_game_3_objs, plot_3d_tradeoff, 
                                 runtime_hvol, plot_runtime)
import matplotlib.pyplot as plt
import time

ImportError: cannot import name 'Problem' from 'platypus' (C:\Users/lbl59/AppData/Local/Programs/Python/Python39/Lib/site-packages\platypus\__init__.py)

In [10]:
import ipywidgets as widgets
# Based on Hadjimichael et al 2020
prob_formulation = widgets.Dropdown(options=['5-objective', '3-objective'],
                         description='', value='5-objective', disabled=False)
prob_formulation

Dropdown(options=('5-objective', '3-objective'), value='5-objective')

### Initializing the problem

Define number of decision variables, constraints, and specify problem formulation (3- or 5-objective)

In [11]:
nVars = 6   # Define number of decision variables
nObjs = 5
if prob_formulation == '5-objective':
    nObjs = 5
elif prob_formulation == '3-objective':
    nObjs = 3
nCnstr = 1      # Define number of decision constraints

problem = Problem(nVars, nObjs, nCnstr)     

# set bounds for each decision variable
problem.types[0] = Real(0.0, 1.0)
problem.types[1] = Real(0.0, 1.0)
problem.types[2] = Real(0.0, 1.0)
problem.types[3] = Real(0.0, 1.0)
problem.types[4] = Real(0.0, 1.0)
problem.types[5] = Real(0.0, 1.0)

# all values should be nonzero
problem.constraints[:] = "==0"

# set problem function
if nObjs == 5:
    problem.function = fish_game_5_objs
else:
    problem.function = fish_game_3_objs

algorithm = BorgMOEA(problem, epsilons=0.001)

NameError: name 'Problem' is not defined

### Run the algorithm

First, run the algorithm with a small NFE to make sure things are working. It is also advisable to time the run to gauge the amount of time you will require to run more NFEs.

**Need to include explanations for abbreviations used for the objectives**

In [ ]:
# begin timing the borg run
borg_start_time = time.time()

# begin optimization
algorithm.run(100)

# end timing
borg_end_time = time.time()

# calculate and print total time taken for optimization with PyBorg
borg_total_time = borg_end_time - borg_start_time

print(f"borg_total_time={borg_total_time}s")

The objectives scores arn't very good, but that is because the number of function evaluations is so low. In order to get a better set of solutions, we need to run the MOEA for many function evaluations.  

The next section demonstrates the change in objective performance with respect to the number of function evaluations.


### Plot the tradeoff surface
Here, we plot a 3-dimensional plot showing the tradeoff between a select number of objectives. If you have selected the 5-objective problem formulation, you can select the three objectives you would like to analyze the tradeoff surface for.

In [ ]:
# Plot objective tradeoff surface
fig_objs = plt.figure()
ax_objs = fig_objs.add_subplot(111, projection='3d')

objs_indices = [0, 1, 2]
obj_labels = ['Mean NPV', 'Mean prey deficit', 'Mean WCLH']
obj_min = [-6000, 0, 0]

plot_3d_tradeoff(algorithm, ax_objs, objs_indices, obj_labels, obj_min)

### MOEA Diagnostics

Explain why to do diagnostic tests.

Reed group Alum Prof. [Joe Kasprzyk wrote a post describing three common MOEA diagnostic metrics](https://waterprogramming.wordpress.com/2013/06/25/moea-performance-metrics/): generational distance, epsilon indicator, and hypervolume. 

#### Hypervolume

The hypervolume is a measure of the multi-dimensional volume _dominated_ by the approximated pareto front. As the pareto front advances toward the "_ideal_" solution, this value approaches 1. 

The efficiency of an MOEA in optimizing a solution can be considered by measuring the hypervolume with respect to the number of function evaluations. This allows the user to understand how quickly the MOEA is converging to a good set of solutions, and how many function evaluations are needed to achieve a good set of solutions.
